In [1]:
import os
import pandas as pd
import datetime as dt
import pytz
import time
import numpy as np
from statistics import mean

from uce_resources import get_applied_forecast

In [2]:

sites_list = ['Pohrebyshche', 'Bilashky', 
                'Bar', 'Balky', 'Verkhivka', 'Kopaihorod', 'Stanislavchyk', 'Sharhorod_1', 'Hnatkiv', 'Porohy',
                'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Cherniatka','Chechelnyk_1', 'Chechelnyk_2',
                'Komyshany_1', 'Komyshany_2', 'Bilozerka', 'Myroliubivka', 'Kyselivka',
                'Oleshky_1', 'Oleshky_2', 'Poniativka', 'Kostohryzove', 
                'Veliton', 'Velihen', 
                'Mykolaivka', 'Rubanivka', 'Mala_Lepetykha', 'Kachkarivka', 
                'Liubymivka', 'Vasylivka',
                'Dibrovka', 'Yelanets_1', 'Yelanets_2', 
                'Afanasiivka', 'Novokondakove', 'Bazaltova', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 
                'Teplychna', 
                'Balivka', 'Solone', 'Stepnohirsk', 
                'Kulevcha']

today = dt.datetime.today()#-dt.timedelta(1)

target_year = today.year
target_month = today.month
target_day = today.day
forecasts_types = ['real']

target_folder = 'data/forecasts/factor/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# sites_list = ['Pohrebyshche', 'Kulevcha']


# Data preparation section

In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import DO_URL

engine_source = create_engine(DO_URL, poolclass=NullPool)
metadata_source = MetaData()
metadata_source.reflect(bind=engine_source)


from settings.db import WAREHOSUE_URL

engine_warehouse = create_engine(WAREHOSUE_URL, poolclass=NullPool)

c:\Users\o.babenko\.conda\envs\uce_daily\lib\site-packages\ipykernel_launcher.py:8: SAWarning: Did not recognize type 'point' of column 'location'
  


In [4]:
import importlib
imported_module = importlib.import_module("get_current_production_powermeter")
importlib.reload(imported_module)
from get_current_production_powermeter import OperativeProduction
from settings.apis import BORD_API_SETTINGS

PowermeterDataGetter = OperativeProduction(**BORD_API_SETTINGS)

Token updated!
Site ids updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!


In [5]:
sites_data = list()
raw_sites_data = list()
applied_sites_data = list()

with engine_source.connect() as connection:
        
    for site in sites_list:
        start = time.time()
        # print('-'*50)
        # print(site)
        site_data = dict()

        sites_table = metadata_source.tables['sites']
        list_to_select = [
            sites_table.c.id, 
            sites_table.c.legal_entity, 
            sites_table.c.location, 
            sites_table.c.region, 
            sites_table.c.cluster,
            sites_table.c.installed_capacity_dc,
            sites_table.c.grid_capacity,
            sites_table.c.w_code
            ]
        query = select(list_to_select).where(sites_table.c.displayable_name == site)
        site_id_response = connection.execute(query).fetchall()[0]

        site_id = site_id_response[0]
        legal_entity_id = site_id_response[1]
        location = site_id_response[2]
        region = site_id_response[3]
        cluster = site_id_response[4]
        capacity_dc = site_id_response[5]
        grid_capacity = site_id_response[6]
        w_code = site_id_response[7]
        
        with engine_warehouse.connect() as connection_warehouse:
            query = f"""
            SELECT grid_capacity from dim_site
            WHERE site_name = '{site}';
            """
            response = connection_warehouse.execute(query).fetchall()[0]

        powermeter_data = PowermeterDataGetter.get_data(w_code, today)
        powermeter_data.columns= ['yield']
   

        first_date=today-dt.timedelta(1)
        last_date=today

        applied_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data']
        )
        applied_forecast.columns = ['applied_forecast']

        raw_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data'],
            forecast_type='forecast_applied_raw'
        )
        raw_forecast.columns = ['raw_forecast']

        forecast_data = pd.concat([applied_forecast, raw_forecast,], axis=1, join='outer').reindex(columns=['raw_forecast', 'applied_forecast'])
        forecast_data[f"{site}"] = round(forecast_data['applied_forecast'] / forecast_data['raw_forecast'], 2)
        forecast_data = forecast_data.drop(columns=["raw_forecast", "applied_forecast"])
        # print(forecast_data)

        sites_data.append(forecast_data)
        koef_merged_data = pd.concat(sites_data, axis=1)

        applied_forecast_data = pd.concat([powermeter_data, applied_forecast,], axis=1, join='outer').reindex(columns=['yield', 'applied_forecast'])
        applied_forecast_data[f"{site}"] = round(applied_forecast_data['yield'] / applied_forecast_data['applied_forecast'], 2)
        applied_forecast_data = applied_forecast_data.drop(columns=["yield", "applied_forecast"])
        # print(applied_forecast_data)
        
        applied_sites_data.append(applied_forecast_data)
        applied_merged_data = pd.concat(applied_sites_data, axis=1)



        raw_forecast_data = pd.concat([powermeter_data, raw_forecast,], axis=1, join='outer').reindex(columns=['yield', 'raw_forecast'])
        raw_forecast_data[f"{site}"] = round(raw_forecast_data['yield'] / raw_forecast_data['raw_forecast'], 2)
        raw_forecast_data = raw_forecast_data.drop(columns=["yield", "raw_forecast"])
        # print(raw_forecast_data)
        
        raw_sites_data.append(raw_forecast_data)
        raw_merged_data = pd.concat(raw_sites_data, axis=1)


        end = time.time()

        print('{}: ок! Processing took {} seconds'.format(site, round(end - start, 2)))

    koef_merged_data.index=koef_merged_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%d %H:%M')
    applied_merged_data.index=applied_merged_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%d %H:%M')
    raw_merged_data.index=raw_merged_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%d %H:%M')


    # file_name = f'forecast_today_{target_year}_{target_month}_{target_day}.xlsx'  
    # file_path = os.path.join(target_folder, file_name)

    # with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    #     merged_data.to_excel(writer, sheet_name='Coefficient')
    #     raw_merged_data.to_excel(writer, sheet_name='Raw_forecast')
    #     applied_merged_data.to_excel(writer, sheet_name='Applied_forecast')
        



Data read successfully: 33, 34
Pohrebyshche: ок! Processing took 3.2 seconds
Data read successfully: 34, 29
Bilashky: ок! Processing took 1.89 seconds
Data read successfully: 5, 922
Bar: ок! Processing took 1.65 seconds
Data read successfully: 7, 921
Balky: ок! Processing took 1.79 seconds
Data read successfully: 6, 902
Verkhivka: ок! Processing took 1.7 seconds
Data read successfully: 13, 939
Data read successfully: 13, 940
Kopaihorod: ок! Processing took 3.04 seconds
Data read successfully: 12, 953
Stanislavchyk: ок! Processing took 1.79 seconds
Data read successfully: 8, 712
Data read successfully: 8, 717
Sharhorod_1: ок! Processing took 4.13 seconds
Data read successfully: 53, 604
Data read successfully: 53, 610
Hnatkiv: ок! Processing took 3.68 seconds
Data read successfully: 51, 671
Data read successfully: 51, 677
Porohy: ок! Processing took 3.21 seconds
Data read successfully: 16, 794
Hlybochok_1: ок! Processing took 2.46 seconds
Data read successfully: 17, 796
Hlybochok_2.1: ок

In [6]:
sites_data = list()

with engine_source.connect() as connection:
        
    for site in sites_list:
        start = time.time()
        # print('-'*50)
        # print(site)
        site_data = dict()

        sites_table = metadata_source.tables['sites']
        list_to_select = [
            sites_table.c.id, 
            sites_table.c.legal_entity, 
            sites_table.c.location, 
            sites_table.c.region, 
            sites_table.c.cluster,
            sites_table.c.installed_capacity_dc,
            sites_table.c.grid_capacity,
            sites_table.c.w_code
            ]
        query = select(list_to_select).where(sites_table.c.displayable_name == site)
        site_id_response = connection.execute(query).fetchall()[0]

        site_id = site_id_response[0]
        legal_entity_id = site_id_response[1]
        location = site_id_response[2]
        region = site_id_response[3]
        cluster = site_id_response[4]
        capacity_dc = site_id_response[5]
        grid_capacity = site_id_response[6]
        w_code = site_id_response[7]
        
        with engine_warehouse.connect() as connection_warehouse:
            query = f"""
            SELECT grid_capacity from dim_site
            WHERE site_name = '{site}';
            """
            response = connection_warehouse.execute(query).fetchall()[0]
            
        grid_capacity = response[0]
        latitude, longitude = map(float, location.replace('(', '').replace(')', '').split(','))

        inverters_data = PowermeterDataGetter.get_data(w_code, today)
        mms_data = inverters_data

        first_date=today-dt.timedelta(1)
        last_date=today

        applied_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data']
        )
        applied_forecast.columns = ['forecast']

        raw_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data'],
            forecast_type='forecast_applied_raw'
        )
        raw_forecast.columns = ['raw_forecast']

        forecast = applied_forecast
        forecast_data = pd.concat([forecast, raw_forecast, mms_data.loc[mms_data.index >= forecast.index.min()]], axis=1, join='outer').reindex(columns=['yield', 'raw_forecast', 'forecast'])

        site_series = pd.Series(index=forecast_data.index, data=site)
        latitude_series = pd.Series(index=forecast_data.index, data=latitude)
        longitude_series = pd.Series(index=forecast_data.index, data=longitude)
        region_series = pd.Series(index=forecast_data.index, data=region)
        cluster_series = pd.Series(index=forecast_data.index, data=cluster)
        capacity_dc_series = pd.Series(index=forecast_data.index, data=capacity_dc)
        grid_capacity_series = pd.Series(index=forecast_data.index, data=grid_capacity)

        site_data = pd.concat([
                    site_series, 
                    latitude_series, 
                    longitude_series,
                    region_series, 
                    cluster_series,
                    capacity_dc_series,
                    grid_capacity_series
                ], 
                axis=1
            )
        site_data.columns = ['site', 'latitude', 'longitude', 'region', 'cluster', 'capacity_dc', 'grid_capacity']                       
        site_data['date'] = site_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%d')
        site_data['hour'] = site_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%H:%M')
        
        site_data = pd.concat([site_data, forecast_data], axis=1)

        site_data['factor'] = round(site_data['forecast'] / site_data['raw_forecast'], 2)
        forecast_accuracy=(site_data['yield'] / site_data['forecast'])*100
        site_data['forecast_accuracy'] = round(forecast_accuracy)
        raw_forecast_accuracy=(site_data['yield'] / site_data['raw_forecast'])*100
        site_data['raw_forecast_accuracy'] = round(raw_forecast_accuracy)
        
        sites_data.append(site_data)
        end = time.time()
       
        print('{}: ок! Processing took {} seconds'.format(site, round(end - start, 2)))

        data = pd.concat(sites_data, ignore_index=True).fillna(0)
        data = data.drop_duplicates(keep='first')

Data read successfully: 33, 34
Pohrebyshche: ок! Processing took 2.07 seconds
Data read successfully: 34, 29
Bilashky: ок! Processing took 2.15 seconds


OperationalError: (psycopg2.OperationalError) connection to server at "db-postgresql-do-pv-do-user-1646238-0.db.ondigitalocean.com" (134.122.79.59), port 25060 failed: FATAL:  remaining connection slots are reserved for non-replication superuser connections

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
file_name = f'forecast_today_{target_year}_{target_month}_{target_day}.xlsx'  
file_path = os.path.join(target_folder, file_name)

with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    koef_merged_data.to_excel(writer, sheet_name='Coefficient')
    raw_merged_data.to_excel(writer, sheet_name='Raw_forecast')
    applied_merged_data.to_excel(writer, sheet_name='Applied_forecast')
    data.to_excel(writer, sheet_name='all_sites')